# Experiment C 

Ideas:

~~1) Estudar as diferentes distâncias para vetores 1-D em `scipy.spatial.distance`.~~

~~2) De 1, implementar e otimizar os que melhor performam comos portfólios.~~

3) Ver a possibilidade de recomendar com base em 2 ou mais empresas (`users`)

Eu não queria usar valores médios entre os vetores ou representantes de clusters,
pois creio que a distorção espacial ao calcular um vetor médio pode ser maior quanto
mais empresas eu tiver de entrada. Penso que uma abordagem interessante é que garante
pelo menos uma proximidade maior entre as empresas é por votação. Via clsuter e vetor
médio eu faço recomendações que não são as mais próximas de todos, mas nem tão distantes,
mas isso pode excluir os potenciais melhores recomendações. Com votação, eu aumento as 
chances de recomendar empresas que são muito próximas para umas, ainda que seja distante
de outras. 

4) Validar as recomendações com os portfólios.

Como utilizar os portfólios dados para validar o modelo? Não sabe-se a ordem em que os
portfólios foram crescendo. Então fazer um-a-um e N-a-N. Recomendações 1-para-1, 1-para-N, N-para-1 e N-para-N.

Estou pensando em uma métrica que trate de forma acumulada, p.e.:

- **A) 1-para-1:** Para cada uma empresa dentro de cada portfólio, eu vejo se a recomendação já está dentro do portfólio.

- **B) 1-para-N:** Para cada uma empresa dentro de cada portfólio, requisito N recomendações. Verifico quantas das N recomendações está no portfólio.

- **C) N-para-1:** Amostro N empresas e requisito 1 recomendação e confiro se está dentro do portfólio. Repito isso uma M vezes.

- **D) N-para-N:** Amostro N empresas e requisito N recomendação e confiro se está dentro do portfólio. Repito isso uma M vezes. 

4.1) E o desempate? Pensando...

Ideia: um misto entre o ranqueamento individual (1-para-N), mais votados e sorteio aleatório dos empatados por último.

Como abordar isso?

Posso fazer algo iterativo, tendo como objetivo obeter as $k$ recomendações solicitadas. Como o cálculo dos vizinhos é feito só uma vez, processamento não será um problema.

- **a)** Pego $L \times \text{div}(k,N_{in}) + \min(\text{mod}(k,N_{in}),1)$ recomendações por usuário de entrada.
- **b)** Ordeno por votação e desempato por ordem de proximidade individual.
- **c)** Se a lista ficar curta, faço $L \leftarrow L+1$ e volto para **a** até cehgar a $k$ recomendações.



*Author: Israel Oliveira [\[e-mail\]](mailto:'Israel%20Oliveira%20'<prof.israel@gmail.com>)*

In [1]:
%load_ext watermark

In [58]:
from typing import NewType, List
import functools
import operator

import numpy as np
import pandas as pd
from surprise import SVD, accuracy, Dataset, Reader
from scipy.spatial.distance import cosine

from loguru import logger
from tqdm import tqdm

In [23]:
# Run this cell before close.
%watermark
%watermark -p loguru
%watermark -p scipy
%watermark -p surprise
%watermark --iversion
%watermark -b -r -g

2020-06-22T02:10:13+00:00

CPython 3.7.7
IPython 7.15.0

compiler   : GCC 8.3.0
system     : Linux
release    : 5.4.0-7626-generic
machine    : x86_64
processor  : 
CPU cores  : 8
interpreter: 64bit
loguru 0.5.1
scipy 1.5.0
surprise 0.1
numpy  1.19.0
pandas 1.0.5

Git hash: 1b3648710901d000d99c99652b7708a74d60ed4c
Git repo: https://github.com/ysraell/aceleradev_private.git
Git branch: master


In [4]:
path_data = '../data/'
top_cols = pd.read_csv('top_cols.csv')['cols'].to_list()
df_marked = pd.read_csv(path_data+'estaticos_market.csv', usecols=top_cols)
col_user = 'id'
top_cols.remove(col_user)

rest_cols = []
for col in top_cols:
    df_marked[col] = df_marked[col].fillna(0)*1

def normalize(x):
    return (x-np.min(x))/(np.max(x) - np.min(x)) if (np.max(x) - np.min(x)) > 0 else (x-np.min(x))

escala = 100
for col in top_cols:
    df_marked[col] = (escala*normalize(df_marked[col].tolist())).astype(np.uint8)
    
remove_cols = []
for col in top_cols:
    if df_marked[col].nunique() == 1:
        remove_cols.append(col)

df_marked = df_marked.drop(columns=remove_cols)
for col in remove_cols:
    top_cols.remove(col)

df_marked = pd.melt(df_marked, id_vars=["id"], var_name="itemID", value_name="rating").rename(columns={"id": "userID"})

reader = Reader(rating_scale=(0, escala))
#data = Dataset.load_from_df(df_marked[['userID', 'itemID', 'rating']].sample(frac=0.2), reader)
data = Dataset.load_from_df(df_marked[['userID', 'itemID', 'rating']], reader)
del df_marked

df_ep_list = [pd.read_csv(path_data+'estaticos_portfolio{}.csv'.format(i+1)) for i in range(3)]
tmp = []
for i in range(3):
    df_ep_list[i]['P'] = i+1 
    tmp.append(df_ep_list[i][['id','P']])
df_ep = pd.concat(tmp)
del df_ep_list
del tmp

In [59]:
Uid = NewType('uid', int)
Raw = NewType('raw', str)

def flat(a):
    return functools.reduce(operator.iconcat, a, [])

class ExSVD(SVD):
    """
        Classe extendida da surprise.SVD.
        
    
    """
    
    def __init__(self,**args):
        self.matrix_dict = {}
        super().__init__(**args)

    def fit(self,trainset: Dataset):
        """
            Reimplementei a SVD.fit para colocar um logger nível INFO.
        """
        logger.info("Treinando modelo SVD...")
        super().fit(trainset)
        logger.info("Pronto!")
    
    def _get_neighbors(self,uid: Uid, k: int = 1, black_list: List[Uid] = []):
        """
            Calcula todas as distâncias entre 'uid' de entrada e todos os outros 'uid'.
            A distância calciulada é armazenda e não calculada novamente. 
        """
        black_list.append(uid)
        k = k if k >= 0 else 0
        logger.info("Calculando todos os vizinhos...")
        for uid2 in tqdm(self.trainset.all_users()):
            ordered = tuple(sorted((uid,uid2)))
            if (uid2 not in black_list) and (ordered not in self.matrix_dict.keys()):
                self.matrix_dict[ordered] = cosine(self.pu[uid],self.pu[uid2])
        return [x[0] for x in sorted(
            [
                (uid2, self.matrix_dict[tuple(sorted((uid,uid2)))]) 
                for uid2 in self.trainset.all_users()
                if (uid2 not in black_list)
            ], key=lambda x: x[1])][:k-1]

    def _uid2raw(self, uid: Uid)-> str:
        '''
            uid -> raw.
            Valor interno para externo, o nome original do usuário.
        '''
        return self.trainset.to_raw_uid(uid)
    
    def _raw2uid(self, raw: Raw)-> int:
        '''
            raw -> uid.
            Valor externo para interno, o id interno do usuários..
        '''
        return self.trainset.to_inner_uid(raw)
    
    

In [44]:
ex_algo = ExSVD(n_factors=5, verbose=True)
ex_algo.fit(data.build_full_trainset())

2020-06-22 03:09:04.401 | INFO     | __main__:fit:19 - Treinando modelo SVD...


Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


2020-06-22 03:11:41.276 | INFO     | __main__:fit:21 - Pronto!


In [48]:
ex_algo._get_neighbors(1,10)

2020-06-22 03:17:04.062 | INFO     | __main__:_get_neighbors:30 - Calculando todos os vizinhos...
100%|██████████| 462298/462298 [00:00<00:00, 654952.29it/s]


[75, 74, 67, 41, 24, 37, 363, 88, 89]

In [61]:
def recomender(self, in_list: List[Raw], k: int = 1, L: int = 3, limit: int = 10)-> List[Raw]:
    '''
        Faz as recomendacoes.
        ##### Função incompleta #####
    '''
    # Pega quantas recomendações por usuário em `in_list`,
    # mas sem deixar faltar
    N_in = len(in_list)
    k = k if k > 0 else 1
    R_per_in = L*(k//N_in + min(k%N_in,1))
    
    uid_in_list = []
    for raw in in_list:
        uid_in_list.append(self._raw2uid(raw))

    done = False
    flag = True
    Rounds = 0
    while limit and (not done):
        Rounds += 1
        # Ele sempre pega todos novamente.
        recomendations_list = []
        for i,uid in enumerate(uid_in_list):
            logger.info("Calculando todos os vizinhos...{:,}/{:,} (Round: {:,}).".format(i+1,N_in,Rounds))
            recomendations_list.append(self._get_neighbors(uid,R_per_in,in_list))
        # Quando limit = 0, encerra.
        limit -= 1
        # Quando tem gente o suficiente, encerra.
        if len(set(flat(recomendations_list))) >= k:
            done = True
        # Depois do primeiro loop, pega um a mais.
        R_per_in += 1
    
    # Agora eu preciso rodar isso para ver como posso seguir.
    return recomendations_list
            
    

In [62]:
recomender(ex_algo,df_ep['id'].head(2).tolist())

2020-06-22 03:25:51.084 | INFO     | __main__:recomender:23 - Calculando todos os vizinhos...1/2 (Round: 1).
2020-06-22 03:25:51.085 | INFO     | __main__:_get_neighbors:30 - Calculando todos os vizinhos...
100%|██████████| 462298/462298 [00:00<00:00, 645036.69it/s]
2020-06-22 03:25:52.710 | INFO     | __main__:recomender:23 - Calculando todos os vizinhos...2/2 (Round: 1).
2020-06-22 03:25:52.710 | INFO     | __main__:_get_neighbors:30 - Calculando todos os vizinhos...
100%|██████████| 462298/462298 [00:00<00:00, 632462.79it/s]


[[148, 367], [339312, 610]]